<a href="https://colab.research.google.com/github/edu-sabino/Python-Practices/blob/main/Webscraping_Stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install requests-html requests

In [40]:
import requests
from requests_html import HTML
import time
import pandas as pd

In [41]:
def parse_tagged_page(html):
  question_summaries = html.find('.s-post-summary')
  key_names = ['id', 'question', 'date', 'tags', 'answers']
  data_needed = ['.data-post-id', '.s-link', '.s-user-card--time', '.s-post-summary--meta-tags', '.s-post-summary--stats-item']
  dados = []

  for q_element in question_summaries:
    question_data = {}
    for i, head in enumerate(data_needed):
      keyname = key_names[i]
      if head == '.data-post-id':
        sub_el = q_element.attrs['data-post-id'] # ex: 70574795
        question_data[keyname] = sub_el
     
      elif head == '.s-post-summary--stats-item':
        sub_el = q_element.find(head)
        question_data[keyname] = sub_el[1].text
     
      elif head == '.s-user-card--time':
        sub_el = q_element.find(head, first = True).text[6:12]
        question_data[keyname] = sub_el
      else:
        sub_el = q_element.find(head, first = True).text
        question_data[keyname] = sub_el
     #print(sub_el.text)
    dados.append(question_data)
  return dados

In [42]:
def extract_data_from_url(url):
  r = requests.get(url) # usando o método GET para armazenar a página numa variável
  if r.status_code not in range(200, 299):
    return []
  html_str = r.text # exibindo o texto do HTML
  html = HTML(html=html_str) # Instanciando um objeto 'html' da classe HTML
  dados = parse_tagged_page(html)
  return dados

In [43]:
def scrape_tag(tag='python', filter='newest', page=3230, max_pages=50):
  base_url = 'https://stackoverflow.com/questions/tagged/'
  dados = []
  for p in range(max_pages):
    page = 3230 + p # a medida que o tempo passa, este número vai aumentando
    url = f"{base_url}{tag}?tab={filter}&page={page}&pagesize=50"
    dados += extract_data_from_url(url)
    time.sleep(1)
  return dados

In [44]:
dados_python_22 = scrape_tag()
dados_python_22
#{'answers': '0 answers',
#  'date': 'Jan 13',
#  'id': '70703241',
#  'question': 'How to add custom number of particles using Ocean Parcels Python Module',
#  'tags': 'python'},

[{'answers': '0 answers',
  'date': 'Jan 15',
  'id': '70719745',
  'question': 'Discord 20001 Bots cannot use this endpoint',
  'tags': 'python oauth-2.0 get request discord'},
 {'answers': '2 answers',
  'date': 'Jan 15',
  'id': '70719741',
  'question': 'How to create a function inside the class which can tell maximum of an attribute (age) and its corresponding attribute to this max age (i.e. name)?',
  'tags': 'python class'},
 {'answers': '0 answers',
  'date': 'Jan 15',
  'id': '70719725',
  'question': 'aiogram bot stops answering messages several minutes after the bot was run',
  'tags': 'python asynchronous telegram-bot aiogram'},
 {'answers': '0 answers',
  'date': 'Jan 15',
  'id': '70719717',
  'question': 'Issue moving object in pygame affecting background',
  'tags': 'python pygame'},
 {'answers': '1 answer',
  'date': 'Jan 15',
  'id': '70719703',
  'question': 'Is there anyway command to round off the numbers in list of the entire Jupyter notebook globally with out usi

In [54]:
ids_list = [] # lista onde será armazenado os ids das perguntas com apenas uma resposta
for question in dados_python_22:
  if question['answers'] ==  '1 answer':
    #print(question['id'])
    single_id = question['id']
    ids_list.append(single_id)
  else:
    continue
print(len(ids_list))

1218


In [59]:
def scrape_answers():
  base_url = 'https://stackoverflow.com/questions/'
  respostas = []
  for i in range(1000):
    url = f"{base_url}{ids_list[i]}"
    respostas += extract_answers_from_url(url) # definir função para extrair só as respostas
    time.sleep(0.5)
  return respostas

def extract_answers_from_url(url):
  r = requests.get(url)
  if r.status_code not in range(200, 299):
   return []
  html_str = r.text # exibindo o texto do HTML
  html = HTML(html=html_str) # Instanciando um objeto 'html' da classe HTML
  respostas = parse_question_page(html)
  return respostas

def parse_question_page(html):
  question_answers = html.find('.answer') # entrega uma lista com os elementos dentro da tag <div> com essa classe
  cabecalhos = ['id_q', 'id_a', 'text', 'user']
  classes_necessarias = ['.data-parentid', '.data-answerid', '.s-prose', '.d-none']
  respostas = []

  for a_element in question_answers:
    answer_data = {}
    for i, head in enumerate(classes_necessarias):
      cabecalho = cabecalhos[i]
      if head == '.data-parentid':
        sub_el = a_element.attrs['data-parentid'] # ex: 70574795
        answer_data[cabecalho] = sub_el
     
      elif head == '.data-answerid':
        sub_el = a_element.attrs['data-answerid'] # ex: 70574795
        answer_data[cabecalho] = sub_el
        #sub_el = q_element.find(head)
        #question_data[keyname] = sub_el[1].text
     
      elif head == '.s-prose': # achar os textos com a tag <p> e concatená-los em uma única string
        sub_el = a_element.find(head, first=True).text # retorna uma lista com os textos de cada <p>
        answer_data[cabecalho] = sub_el
        #texto_junto = ''
        #for t in sub_el:
          #texto_solto = sub_el[t]
          #texto_junto += str(texto_solto) + ''
        #answer_data[cabecalho] = texto_junto
      else:
        sub_el = a_element.find(head, first = True).text
        answer_data[cabecalho] = sub_el
     #print(sub_el.text)
    respostas.append(answer_data)
  return respostas

In [60]:
answers_python_22 = scrape_answers()
print(len(answers_python_22))

113


In [61]:
df1 = pd.DataFrame(answers_python_22)
df1

,id_q,id_a,text,user
0,70719703,70719730,Is this what you’re looking for?\nlist(map(rou...,
1,70719701,70719736,"You can try:\nA.reshape(2, 2)@x\nIt gives:\nar...",bb1
2,70719691,70719773,"You can try to see the exception message, it s...",pppig
3,70719689,70720888,The image is kind of blurry and this can happe...,
4,70719673,70719822,To return more than one context variables you ...,
...,...,...,...,...
108,70685857,70686573,Proposed solution below. It uses grouping by '...,NickHilton
109,70685851,70685988,"That is because F and A are floats, not iterab...",sarema
110,70685849,70686077,"You don't say what is the desired output, but ...",
111,70685826,70733480,"iter_download is the correct way, however, you...",Lonami


In [62]:
df1.to_excel("113_answers_Jan_22.xlsx", index = False)

In [50]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [51]:
df = pd.DataFrame(dados_python_22)
df

,id,question,date,tags,answers
0,70719745,Discord 20001 Bots cannot use this endpoint,Jan 15,python oauth-2.0 get request discord,0 answers
1,70719741,How to create a function inside the class whic...,Jan 15,python class,2 answers
2,70719725,aiogram bot stops answering messages several m...,Jan 15,python asynchronous telegram-bot aiogram,0 answers
3,70719717,Issue moving object in pygame affecting backgr...,Jan 15,python pygame,0 answers
4,70719703,Is there anyway command to round off the numbe...,Jan 15,python list arraylist,1 answer
...,...,...,...,...,...
2495,70669974,indexing multidimensional array in python,Jan 11,python indexing,2 answers
2496,70669971,How to combine the list of dictionaries using ...,Jan 11,python arrays list sorting dictionary,2 answers
2497,70669947,Can someone please clarify what happens in _ge...,Jan 11,python pytorch dataset huggingface-transformers,2 answers
2498,70669940,How to set query parameters in URL in Django,Jan 11,python api django-rest-framework,1 answer


In [52]:
df.to_csv("python_10question_jan_22_pt7.csv", index=False)